1. Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
import numpy as np
from PIL import Image, ImageTk
import os
import random
import cv2
import tkinter as tk
from tkinter import messagebox


2. Define and Train CNN Model

In [ ]:
# 1. Define and Train CNN Model (Load dataset or retrain if needed)
def create_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


3. Load Data with ImageDataGenerator

In [ ]:
# Load data with ImageDataGenerator
data_dir = 'dataset'
img_height, img_width = 64, 64

train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
import matplotlib.pyplot as plt

# Visualize a batch of augmented images
sample_batch = next(train_generator)
fig, ax = plt.subplots(1, 5, figsize=(15, 5))
for i in range(5):
    ax[i].imshow(sample_batch[0][i])
    ax[i].axis('off')
plt.suptitle("Augmented Training Images")
plt.show()


4. Create Model, Train, and Save


In [ ]:
# Create and train model
model = create_model((img_height, img_width, 3), num_classes=len(train_generator.class_indices))
model.fit(train_generator, epochs=5, validation_data=validation_generator)

model.save('captcha_model.keras')
model = tf.keras.models.load_model('captcha_model.keras')
class_indices = train_generator.class_indices
label_map = {v: k for k, v in class_indices.items()}


# After training the model (inside "4. Create Model, Train, and Save")
history = model.fit(train_generator, epochs=5, validation_data=validation_generator)

# Plot training and validation accuracy/loss
plt.figure(figsize=(12, 4))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()

plt.show()



5. Load Object Images for CAPTCHA

In [ ]:
# 2. Function to load object images
def load_object_images():
    object_images = []
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                try:
                    img = Image.open(img_path).resize((img_width, img_height))
                    img_np = np.array(img)
                    object_images.append((img_np, class_name))
                except Exception as e:
                    print(f"Error loading image {img_name} in class {class_name}: {e}")
    return object_images


6. Define Image Transformation Functions

In [ ]:
# Image transformation functions for CAPTCHA difficulty
def add_noise(img):
    row, col, ch = img.shape
    mean = 0
    sigma = 0.1
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy_img = img + gauss * 255
    noisy_img = np.clip(noisy_img, 0, 255).astype(np.uint8)
    return noisy_img

def apply_blur(img):
    return cv2.GaussianBlur(img, (5, 5), 0)

def adjust_brightness_contrast(img):
    alpha = random.uniform(0.5, 1.5)
    beta = random.randint(-30, 30)
    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    return adjusted

def rotate_image(img):
    angle = random.choice([10, -10, 15, -15])
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, matrix, (w, h))
    return rotated

def transform_image(img_np):
    transformations = [add_noise, apply_blur, adjust_brightness_contrast, rotate_image]
    transformed_img = img_np.copy()
    for transform in random.sample(transformations, k=2):
        transformed_img = transform(transformed_img)
    return transformed_img




7. Generate CAPTCHA Images

In [ ]:
# CAPTCHA generation with transformations
def generate_captcha_images(object_images, num_images=9, target_class="car"):
    target_images = [img for img, label in object_images if label == target_class]
    non_target_images = [img for img, label in object_images if label != target_class]
    
    if len(target_images) < num_images // 2 or len(non_target_images) < num_images // 2:
        print("Not enough images in dataset to create CAPTCHA.")
        return []

    selected_images = random.sample(target_images, k=num_images // 2) + random.sample(non_target_images, k=(num_images - num_images // 2))
    random.shuffle(selected_images)

    captcha_images = []
    for img in selected_images:
        transformed_img = transform_image(img)
        label = target_class if any(np.array_equal(img, target_img) for target_img in target_images) else "non-target"
        captcha_images.append((transformed_img, label))
    
    return captcha_images


8. Define CAPTCHA App GUI

In [ ]:
class CAPTCHAApp:
    def __init__(self, root, captcha_images, target_class):
        self.root = root
        self.root.title("CAPTCHA Verification")
        self.captcha_images = captcha_images
        self.target_class = target_class
        self.selected_indices = []  # Store selected images' indices
        self.current_round = 1
        self.attempts = 0

        self.buttons = []  # Store button objects
        self.frames = []   # Store frame objects for borders

        # Create buttons for images and add to GUI
        self.create_image_buttons()

        # Add label and submit button
        self.pattern_label = tk.Label(self.root, text=f"Select all the {target_class}s", font=('Helvetica', 12, 'bold'))
        self.pattern_label.grid(row=len(captcha_images) // 3 + 1, column=0, columnspan=3)

        self.submit_button = tk.Button(self.root, text="Submit", command=self.submit)
        self.submit_button.grid(row=len(captcha_images) // 3 + 2, column=0, columnspan=3)

    def create_image_buttons(self):
        # Create buttons for the CAPTCHA images
        self.buttons.clear()  # Clear any existing buttons
        self.frames.clear()  # Clear any existing frames
        for idx, (img_np, label) in enumerate(self.captcha_images):
            img_pil = Image.fromarray(img_np)
            img_tk = ImageTk.PhotoImage(img_pil)
            
            # Create a frame for each button
            frame = tk.Frame(self.root, relief="solid", bd=3, padx=5, pady=5)
            frame.grid(row=idx // 3, column=idx % 3, padx=10, pady=10)
            
            btn = tk.Button(frame, image=img_tk, command=lambda idx=idx, frame=frame: self.select_image(idx, frame))
            btn.image = img_tk  # Keep reference to image object to prevent garbage collection
            btn.grid(row=0, column=0)
            self.buttons.append(btn)
            self.frames.append(frame)  # Add frame to the list

    def select_image(self, idx, frame):
        # Toggle selection status of the image
        if idx in self.selected_indices:
            self.selected_indices.remove(idx)
            frame.config(bg="white")  # Reset border color to white when deselected
        else:
            self.selected_indices.append(idx)
            frame.config(bg="green")  # Set border color to green when selected

    def submit(self):
        # Verify the CAPTCHA
        if self.verify_captcha(self.selected_indices, self.captcha_images, self.target_class):
            if self.current_round == 1:
                messagebox.showinfo("Success", "You passed the CAPTCHA for cars! Now select all the bikes.")
                self.current_round = 2
                self.target_class = "bike"
                self.update_captcha_images(reset_selection=True)  # Refresh images for next pattern
            elif self.current_round == 2:
                messagebox.showinfo("Success", "You passed the CAPTCHA for bikes! Now select all the trains.")
                self.current_round = 3
                self.target_class = "train"
                self.update_captcha_images(reset_selection=True)  # Refresh images for next pattern
            elif self.current_round == 3:
                messagebox.showinfo("Success", "You passed the CAPTCHA for trains! Now select all the buses.")
                self.current_round = 4
                self.target_class = "bus"
                self.update_captcha_images(reset_selection=True)  # Refresh images for next pattern
            else:
                messagebox.showinfo("Success", "You are a Human!")
                self.root.quit()  # Exit the Tkinter event loop
                self.root.destroy()  # Close the Tkinter window
        else:
            self.attempts += 1
            if self.attempts >= 3:
                messagebox.showerror("Error", "Too many incorrect attempts. Try again later.")
                self.root.quit()  # Exit the Tkinter event loop
                self.root.destroy()  # Close the Tkinter window
            else:
                messagebox.showerror("Error", "Incorrect selection, please try again.")
            
            # Reset selected indices after wrong attempt
            self.selected_indices = []  
            
            # Refresh the images and apply full border on selected ones
            self.update_captcha_images(reset_selection=True)

    def update_captcha_images(self, reset_selection=False):
        # Generate new set of CAPTCHA images based on the current target class
        self.captcha_images = generate_captcha_images(object_images, num_images=9, target_class=self.target_class)
        self.create_image_buttons()  # Create new buttons for the updated images

        if reset_selection:
            self.reset_image_selection()

        # Update the label with the new target class
        self.pattern_label.config(text=f"Select all the {self.target_class}s")

    def reset_image_selection(self):
        # Reset all selected images to have no border color
        for frame in self.frames:
            frame.config(bg="white")  # Remove any border color
        self.selected_indices = []  # Clear the selected indices

    def verify_captcha(self, selected_indices, captcha_images, target_class):
        # Verify that all selected images belong to the target class and match the correct count
        selected_labels = [captcha_images[i][1] for i in selected_indices]
        if all(label == target_class for label in selected_labels):
            target_images_in_captcha = [i for i, (img, label) in enumerate(captcha_images) if label == target_class]
            if len(selected_indices) == len(target_images_in_captcha):
                return True
        return False


9. Start the CAPTCHA App

In [ ]:
# Load the object images
object_images = load_object_images()

# Generate CAPTCHA images for "car" class
captcha_images = generate_captcha_images(object_images, num_images=9, target_class="car")

# Create and run the CAPTCHA application
root = tk.Tk()
app = CAPTCHAApp(root, captcha_images, target_class="car")
root.mainloop()
